<a href="https://colab.research.google.com/github/ruandonino/scrap-by-XPath/blob/main/ScraperEcommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%pip install requests
#%pip install beautifulsoup4
%pip install scrapy

In [ ]:
from bs4 import BeautifulSoup
import requests

custom_headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Accept-Language': 'da, en-gb, en',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Referer': 'https://www.google.com/'
}

page = requests.get(url = "https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/", headers=custom_headers)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-GB">
 <head>
  <meta charset="utf-8"/>
  <!--SEO Meta Tags-->
  <title>
   AVE /'a:vei/- Ergonomic Wireless (2.4 GHz) Mouse ST304020
  </title>
  <meta content="Standivarius AVE is an ergonomic wireless mouse. It has been designed to provide an ergonomically correct working position to reduce wrist and forearm strain. Its improved grip promotes whole arm movement using large muscle groups of the upper arm and shoulder.Due to its small weight, long battery life, portability and reliable connection, AVE is an ideal part of your mobile ergonomic workstation for hybrid work." name="description"/>
  <meta content="Langstane" name="application-name">
   <meta content="gSV3l7SJQN97HgPAV7P0ih93uygQc2WWziphDpHo" name="csrf-token"/>
   <meta content="JnMNiwKvKDYknFlfD3pgPRoIAv9PC14xWYu6iJrxsFxkDTDGTtYe42QZp6b6XofG" name="idempotency-token"/>
   <meta content="" name="keywords"/>
   <meta content="£" name="currency"/>
   <meta content="GBP" name="currency_iso"/>
   

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
import re  # For cleaning the price
import csv  # For reading URLs and XPaths from CSV and saving the results

# Define the spider class
class PriceSpider(scrapy.Spider):
    name = "price_spider"

    '''
    custom_settings = {
        'DEFAULT_REQUEST_HEADERS': {
            "accept": "*/*",
            "accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7",
            "priority": "u=1, i",
            "referer": "https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/",
            "sec-ch-ua": '"Microsoft Edge";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0",
            "x-requested-with": "XMLHttpRequest"
        },
        'DOWNLOAD_DELAY': 2,
        'COOKIES_ENABLED': True,
        'RETRY_ENABLED': True,
        'RETRY_TIMES': 5,  # Number of retries before giving up
        'RETRY_HTTP_CODES': [403, 500, 502, 503, 504],  # Retry on these HTTP errors
        'DOWNLOADER_MIDDLEWARES': {
        'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware': 1,
        },
        'HTTP_PROXY': 'http://20.210.113.32:8123',
    }
    '''
    custom_headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Accept-Language': 'da, en-gb, en',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Referer': 'https://www.google.com/'
    }

    # Initialize the spider with URLs and XPaths from a CSV file
    def __init__(self, *args, **kwargs):
        super(PriceSpider, self).__init__(*args, **kwargs)
        self.urls_and_xpaths = self.load_urls_and_xpaths('input_data.csv')

    # Load URLs and XPaths from a CSV file
    def load_urls_and_xpaths(self, csv_file):
        urls_and_xpaths = []
        with open(csv_file, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
              print(row)
              urls_and_xpaths.append((row['product_name'].strip(), row['url'].strip(), row['xpath'].strip()))
        return urls_and_xpaths

    # Start the requests based on the URLs from the CSV file
    def start_requests(self):
        for product_name, url, xpath in self.urls_and_xpaths:
            yield scrapy.Request(url=url, meta={'xpath': xpath, 'product_name': product_name}, callback=self.parse)

    # Parse the response and extract the price based on the provided XPath
    def parse(self, response):
        xpath = response.meta['xpath']
        product_name = response.meta['product_name']
        raw_price = response.xpath(xpath).get()
        resp = response.xpath(xpath)
        print(raw_price)
        if raw_price:
            # Clean the price (extract only numbers and decimal points)
            price = re.sub(r'[^\d.]', '', raw_price)
        else:
            price = None

        # Print the cleaned price
        print(f"Scraped Price from {response.url}: {price}")

        # Save the price to the CSV file
        print(product_name)
        self.save_to_csv(product_name, response.url, price)

        # Yield the price (for Scrapy output or pipelines if needed)
        yield {
            'url': response.url,
            'price': price
        }

    # Save the URL and price to a CSV file
    def save_to_csv(self, product_name, url, price):
        with open('scraped_prices.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([product_name, url, price])

# Setup the process to run the spider
process = CrawlerProcess(settings={
    "LOG_LEVEL": "DEBUG",  # Ignore logs for now
})

# Start the spider
process.crawl(PriceSpider)
process.start()  # The script will block here until the spider finishes

INFO:scrapy.utils.log:Scrapy 2.11.2 started (bot: scrapybot)


In [ ]:
price = soup.find_all('meta', {'property': 'product:price:amount'})[1]['content']
currency = soup.find_all('meta', {'property': 'product:price:currency'})[1]['content']
print(f"the price is {price} {currency}")

IndexError: list index out of range

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

# Define the spider class
class PriceSpider(scrapy.Spider):
    name = "price_spider"

    def start_requests(self):
        # You can yield multiple requests here if needed
        yield scrapy.Request(url='https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/', callback=self.parse)

    def parse(self, response):
        # Extract the price and currency using XPath
        price = response.xpath('//*[@id="main-content"]/div[1]/div[2]/div[1]/section[2]/div/div[3]/div[5]/span[2]/text()').get()

        # Print the price and currency
        print(f"Price: {price}")
        yield {
            'price': price
        }

# Setup the process to run the spider
process = CrawlerProcess(settings={
    "LOG_LEVEL": "DEBUG",  # Ignore logs for now
})

# Start the spider
process.crawl(PriceSpider)
process.start()  # The script will block here until the spider finishes

INFO:scrapy.utils.log:Scrapy 2.11.2 started (bot: scrapybot)
2024-10-03 17:07:50 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-10-03 17:07:50 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-10-03 17:07:50 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRI

Price: £58.32


Using Input_data.csv

In [ ]:
!pip install scrapy scrapy-selenium selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
import re  # For cleaning the price
import csv  # For reading URLs and XPaths from CSV and saving the results

# Define the spider class
class PriceSpider(scrapy.Spider):
    name = "price_spider"

    '''
    custom_settings = {
        'DEFAULT_REQUEST_HEADERS': {
            "accept": "*/*",
            "accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7",
            "priority": "u=1, i",
            "referer": "https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/",
            "sec-ch-ua": '"Microsoft Edge";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0",
            "x-requested-with": "XMLHttpRequest"
        },
        'DOWNLOAD_DELAY': 2,
        'COOKIES_ENABLED': True,
        'RETRY_ENABLED': True,
        'RETRY_TIMES': 5,  # Number of retries before giving up
        'RETRY_HTTP_CODES': [403, 500, 502, 503, 504],  # Retry on these HTTP errors
        'DOWNLOADER_MIDDLEWARES': {
        'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware': 1,
        },
        'HTTP_PROXY': 'http://20.210.113.32:8123',
    }
    '''
    custom_headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Accept-Language': 'da, en-gb, en',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Referer': 'https://www.google.com/'
    }

    # Initialize the spider with URLs and XPaths from a CSV file
    def __init__(self, *args, **kwargs):
        super(PriceSpider, self).__init__(*args, **kwargs)
        self.urls_and_xpaths = self.load_urls_and_xpaths('input_data.csv')

    # Load URLs and XPaths from a CSV file
    def load_urls_and_xpaths(self, csv_file):
        urls_and_xpaths = []
        with open(csv_file, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
              print(row)
              urls_and_xpaths.append((row['product_name'].strip(), row['url'].strip(), row['xpath'].strip()))
        return urls_and_xpaths

    # Start the requests based on the URLs from the CSV file
    def start_requests(self):
        for product_name, url, xpath in self.urls_and_xpaths:
            yield scrapy.Request(url=url, meta={'xpath': xpath, 'product_name': product_name}, callback=self.parse)

    # Parse the response and extract the price based on the provided XPath
    def parse(self, response):
        xpath = response.meta['xpath']
        product_name = response.meta['product_name']
        raw_price = response.xpath(xpath).get()
        resp = response.xpath(xpath)
        print(raw_price)
        if raw_price:
            # Clean the price (extract only numbers and decimal points)
            price = re.sub(r'[^\d.]', '', raw_price)
        else:
            price = None

        # Print the cleaned price
        print(f"Scraped Price from {response.url}: {price}")

        # Save the price to the CSV file
        print(product_name)
        self.save_to_csv(product_name, response.url, price)

        # Yield the price (for Scrapy output or pipelines if needed)
        yield {
            'url': response.url,
            'price': price
        }

    # Save the URL and price to a CSV file
    def save_to_csv(self, product_name, url, price):
        with open('scraped_prices.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([product_name, url, price])

# Setup the process to run the spider
process = CrawlerProcess(settings={
    "LOG_LEVEL": "DEBUG",  # Ignore logs for now
})

# Start the spider
process.crawl(PriceSpider)
process.start()  # The script will block here until the spider finishes

INFO:scrapy.utils.log:Scrapy 2.11.2 started (bot: scrapybot)
2024-10-03 18:45:09 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-10-03 18:45:09 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-10-03 18:45:09 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRI

{'url': 'https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/', 'product_name': ' Oryx ', 'xpath': ' //*[@id="main-content"]/div[1]/div[2]/div[1]/section[2]/div/div[3]/div[5]/span[2]'}
{'url': 'https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/', 'product_name': ' AVE', 'xpath': ' //*[@id="productaddtocart_146884"]/div[2]/div[1]/div/div/div[1]'}
{'url': 'https://www.globo.com/', 'product_name': ' test', 'xpath': ' //*[@id="destaque"]/div/div/div[3]/div[4]/div/div/div/a/h2'}


INFO:scrapy.middleware:Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2024-10-03 18:45:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
INFO:scrapy.crawler:Overridden settings:
{}
2024-10-03 18:45:09 [scrapy.crawler] INFO: Overridden settings:
{}
INFO:scrapy.middleware:Enabled downloader middlewares:
['scrapy.downloadermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 's

<h2 class="post__title">Argentina vence a França e pega Brasil na final da Copa do Mundo de futsal</h2>
Scraped Price from https://www.globo.com/: 22
test


DEBUG:scrapy.core.engine:Crawled (200) <GET https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/> (referer: None)
2024-10-03 18:45:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/>
{'url': 'https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/', 'price': '58.32'}
2024-10-03 18:45:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/>
{'url': 'https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/', 'price': '58.32'}


<span data-product-price-without-tax class="price price--withoutTax price-section--minor">£58.32</span>
Scraped Price from https://wellworking.co.uk/accessories/laptop-stands/oryx-evo-d-laptop-stand/: 58.32
Oryx


DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/> (referer: None)
2024-10-03 18:45:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/>
{'url': 'https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/', 'price': None}
2024-10-03 18:45:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/>
{'url': 'https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/', 'price': None}
INFO:scrapy.core.engine:Closing spider (finished)
2024-10-03 18:45:12 [scrapy.core.engine] INFO: Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/

None
Scraped Price from https://www.langstane.co.uk/ave-a-vei-ergonomic-wireless-2-4-ghz-mouse-st304020--1/: None
AVE
